## Visualize prediction results
Feeds randomly selected images into a trained model and visualizes the results

In [ ]:
%matplotlib inline

# Create symlinks so can import them here
!ln -s ../scripts/draw_grid.py draw_grid.py
!ln -s ../src/imgcls/config config
!ln -s ../src/imgcls/modeling modeling

In [ ]:
import os
import json
import random
import numpy as np
from matplotlib import pyplot as plt
from draw_grid import draw_pred_vs_true
from config import get_cfg
import modeling
from detectron2.engine import DefaultPredictor
import cv2
import torch

In [ ]:
# Specify paths
data_json = '../data/synthetic/val.json'
config_file = '../src/config.yaml'
model_weights = '../output/second_good_run/model_final.pth'

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(config_file)
cfg.MODEL.WEIGHTS = os.path.abspath(model_weights)
cfg.INPUT.MIN_SIZE_TEST = 800 # apply resizing?
cfg.INPUT.MAX_SIZE_TEST = 800 

predictor = DefaultPredictor(cfg)
# docs here https://detectron2.readthedocs.io/modules/engine.html#detectron2.engine.defaults.DefaultPredictor

with open(data_json, 'r') as json_file:
    dataset_dicts = json.load(json_file)

In [ ]:
# specify how many random images to output
num_preds = 10
for d in random.sample(dataset_dicts, num_preds):
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    output = predictor(img)  # can test with output = torch.rand(16)
    predictions = output['pred'].to("cpu").round().numpy().astype(int)
    truth_labels = np.array(d['label'])
    img = draw_pred_vs_true(img,
                            predictions, 
                            truth_labels)
   
    plt.imshow(img)
    plt.show()

In [ ]:
# clean up symlinks
!rm draw_grid.py modeling config ../src/imgcls/config/config ../src/imgcls/modeling/modeling